# Linked transactions through decryption mixnet

Actors:
* **Alice:** a client that sends a transaction to the mixer
* **Mixer:** a set of nodes that mix input transactions
* **Bob:** a client that receives a transaction from the mixer
* **Regulator:** an entity that verifies compliance

Tools:
* **ElGamal** encryption for linkage parameter
* **Hybrid encryption** (ElGamal + AES) for encryption mixnet (onion encryption)

In [1]:
from elgamal import ElGamal, ElGamalHashed
from point import Point
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from sage.all_cmdline import *
from hybrid import Hybrid
import secrets
from binascii import hexlify

### Generate Regulator's ElGamal keys

In [2]:
regHenc = ElGamalHashed()
regPK = regHenc.getPublicKey()
print('Regulator public key: {}'.format(regPK))

Regulator public key: (16165501176944392564186016876443754158847076933925766793041796675405726455792:19219681901207557929654192488281158869583873627260977722777957480409268092306)


### Generate mixnet keys

In [3]:
n = 5
mixnet = []
mixnetPK = []
for i in range(0, n):
    mixnetHenc = Hybrid()
    mixnet.append(mixnetHenc)
    mixnetPK.append(mixnetHenc.getPublicKey())
    print('--Mixnet({}) PK: \n   {}'.format(i, mixnetPK[i]))

--Mixnet(0) PK: 
   (11962211047773926181831691402477389803249114059214001555815593173338478044881:13844945796150477210085360869717859121533442528124761197645890830263830745868)
--Mixnet(1) PK: 
   (16312699257534033848797376360549863335552993897633646681980141684934698247015:18424146183485718005714790159910971127785408932902904220535394153459905235442)
--Mixnet(2) PK: 
   (10653313274955147664125312347265840135969802134875877841478087382532526238974:8954498403631560421557185197274021840727649785156168925086934891351651899113)
--Mixnet(3) PK: 
   (8509978647586502421078109040135417738351233630276289872009538662048123412939:7838776322521178835687108566654139005017149526220170759952725762893493549)
--Mixnet(4) PK: 
   (20875364747505857805973099416751738517448628491621601505139526606294968158126:21686669803496922130606767730747321907125708139876758603924496387790948045269)


### Generate Alice's parameters

Alice computes $l1$ and $l2 = Enc_{regPK}(l1)$.
Alice sends $Tx_1(l2)$ to the mixer.

In [4]:
address = secrets.randbits(160).to_bytes(20, 'little')
l1 = secrets.randbits(256)
aliceHenc = ElGamalHashed(regPK)
l2_C1p, l2_c2 = aliceHenc.encrypt(l1)

Encrypt


Format data as $m = (addr || l1)$

------------------
TODELETE: Format data as $m = (addr || l2_{C1}.x || l2_{C1}.y || l2_{c2})$.
Note that $l2_{C1}$ is a point in the curve. The message $m$ will be sent (encrypted) to the mixnet, and the mixer will send $Tx_2(addr, l2)$.

In [5]:
#l2_C1p_x = int(l2_C1p.x).to_bytes(32, 'little')
#l2_C1p_y = int(l2_C1p.y).to_bytes(32, 'little')
    
#m = address + l2_C1p_x + l2_C1p_y + l2_c2
m = address + int(l1).to_bytes(32, 'little')

In [6]:
print('--address: \n   {}'.format(hexlify(address)))
print('--l1: \n   {}'.format(hexlify(l1.to_bytes(32, 'little'))))
print('--l2_C1p: \n   {}'.format(l2_C1p))
print('--l2_c2: \n   {}'.format(hexlify(l2_c2)))
print('--m: \n   {}'.format(hexlify(m)))

--address: 
   b'c387b4291a28dc8ffe88eba6a5570dccd917751d'
--l1: 
   b'a8ceaf325d8e11c7c53b1c2252fe94013963417884b9539aa2166394906e3f68'
--l2_C1p: 
   (14747460089051896594581502614109810246128292942268223647803965030423444617398:7451526578465640430529834458858810654003699292113716608075788272833392701268)
--l2_c2: 
   b'6238cfe66e72a7171c5b3788fe4f7f811da7bd0e680d5f3039db7bfb088ca3d1'
--m: 
   b'c387b4291a28dc8ffe88eba6a5570dccd917751da8ceaf325d8e11c7c53b1c2252fe94013963417884b9539aa2166394906e3f68'


### Alice encrypts m for the mixnet

First, initialize the encryption objects with the mixnet public keys. Note that we use hybrid encryption.

In [7]:
aliceMenc = []
for i in range(0, n):
    aliceMenc.append(Hybrid(mixnetPK[i]))

Encrypt
Encrypt
Encrypt
Encrypt
Encrypt


Then, perform a layered encryption of $m$ using the mixnet public keys. Note that the ciphertext $ctx = c_n || c_t || c$, which are the nonce, the verification tag and the ciphertext itself.

In [10]:
encryptedKeys = []
ciphertexts = [m]

for i in range(0, n):
    c, c_t, c_n, c_C1p, c_c2 = aliceMenc[i].encrypt(ciphertexts[i])
    ctx = c_n + c_t + c
    ciphertexts.append(ctx)
    encryptedKeys.append((c_C1p, c_c2))

print('(M{}) ctx: {}'.format(n-1, hexlify(ciphertexts[n-1])))

(M4) ctx: b'5d8d478d877112f8ed218757d02efc32471cb9f728a26735efc237adfa138676a39b62f2fc10f34164fda9fdef97df009c6541eb729a8f1ed654d1d2c44338fe14feb518dfb8671b8cdbca08440e87a29455921e43eb6ee560e55f4e1924aafa47b410f1c9330696855c7e1d1e7478a0f0fea2614df2e09101b050d325ee9c23ab43c7e2c069a7e7f64ca5faf75b3c0713f91152527c2a865fa60ed895f67757ffc6b7680242e024f8d61a44dad848caedd60603'


Alice sends $ctx4$ to $M4$, together with all the symmetric encrypted keys.

### Compute the layered decryption

Each mixnet party deletes a layer of encryption, and performs a shuffling of the ciphertexts (the latter is ommitted in this example).

In [11]:
decryptions = []

for i in range(n, 0, -1):
    c_n = ciphertexts[i][:16]
    c_t = ciphertexts[i][16:32]
    c = ciphertexts[i][32:len(ciphertexts[i])]
    c_encKey = encryptedKeys[i-1]
    
    data = mixnet[i-1].decrypt(c, c_t, c_n, c_encKey[0], c_encKey[1])
    decryptions.append(data)

The last decryption is the message $m$.

In [12]:
new_m = decryptions[n-1]
a = 0
b = 20
mix_addr = new_m[a:b]
a = b
b += 32
mix_l1 = int.from_bytes(new_m[a:b], 'little')

#reg_l2C1x = int.from_bytes(new_m[a:b], 'little')
#a = b
#b += 32
#reg_l2C1y = int.from_bytes(new_m[a:b], 'little')
#a = b
#reg_l2c2 = new_m[a:]

#reg_l2C1 = Point(reg_l2C1x, reg_l2C1y)

print('address: {}'.format(hexlify(mix_addr)))
print('l1: {}'.format(mix_l1))
#print('l2_C1p: {}'.format(reg_l2C1))
#print('l2_c2: {}'.format(hexlify(reg_l2c2)))

address: b'c387b4291a28dc8ffe88eba6a5570dccd917751d'
l1: 47152610707698191413349915212418927833670448422911410738580342833855413079720


The mixer emits $Tx(l1)$

### Regulator verification

Finally, the Regulator can decrypt $l2$ and obtain $l1$.

In [13]:
val = regHenc.decrypt(l2_C1p, l2_c2)
print('l1(reg): {}'.format(hexlify(val)))
print('l1(reg) == l1?: {}'.format(val == mix_l1.to_bytes(32, 'little')))

l1(reg): b'a8ceaf325d8e11c7c53b1c2252fe94013963417884b9539aa2166394906e3f68'
l1(reg) == l1?: True
